In [4]:
%load_ext autoreload
%autoreload 2

import helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
def driver_code():

    count = 0

#     file_path = helper.download_file(
#         "https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv"
#     )
    file_path = '../../Data/Global_Mobility_Report.csv'
    conn = helper._make_connection()
    helper.create_tables_google(conn)

    df = helper.pd.read_csv(filepath_or_buffer=file_path, parse_dates=True, low_memory=False)

    countries = df[["country_region_code", "country_region"]].drop_duplicates()
    stats = df.drop(
        labels=["country_region", "iso_3166_2_code", "census_fips_code", "place_id"],
        axis=1,
    )
    location_util = df[
        ["country_region_code", "iso_3166_2_code", "census_fips_code", "place_id"]
    ].drop_duplicates()

    countries["country_region_code"][countries["country_region"] == "Namibia"] = "NA"

    count += helper.import_data(conn, "countries_google", countries)
    count += helper.import_data(conn, "mobility_stats_google", stats)
    count += helper.import_data(conn, "location_util_google", location_util)
    
    helper.os.remove('../../Data/Global_Mobility_Report.csv')
    
    return count

In [10]:
driver_code()

Connecting to the PostgreSQL database...
Connection successful
Version: ('PostgreSQL 14.0, compiled by Visual C++ build 1914, 64-bit',)
Fetching Data from database...
Tables created successfully!
----------------------------
Truncated countries_google
Done!
-------------------------------
Truncated mobility_stats_google
Done!
-------------------------------
Truncated location_util_google
Done!
-------------------------------


3